In [1]:
# Установка transformers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#Установка catboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
from itertools import groupby

In [4]:
#import необходимых модулей

import pandas as pd
import yaml
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

import numpy as np
from datetime import datetime, timedelta

from catboost import CatBoostClassifier, Pool

In [5]:
#Считывание данных в DataFrame 

train= pd.read_csv('train_predobr_7200_vniz.csv', sep=';', index_col=None)
test= pd.read_csv('test_predobr_do7200_claster.csv', sep=';', index_col=None)
tabliza= pd.read_csv('tabliza_clasters_vniz_5.csv', sep=',', index_col=None)

In [6]:
train

,Номер кластера исходный,Расстояние до центра кластера,overall_worklogs,project_id,Id_razn,count_comments,id,assignee_id,creator_id,summary_to_eng,Year,Month,Day,Hour,DayOfWeek,Text_Summa,claster_centr_ishodn,mean_words_comment,Mesto_in_project
0,1,0.113965,1800,5,0,0,819952,93,93,UI tests commissioned by Dobro KG,2019,10,1,5,2,UI tests commissioned by Dobro KG,2042,0.000000,0.000000
1,0,0.080175,7200,5,0,1,819949,93,93,"UI tests of the ""Profile"" section",2019,10,1,5,2,"UI tests of the ""Profile"" section Trusted and ...",7030,4.000000,0.019231
2,4,0.013658,900,5,0,0,819943,93,93,"UI tests of the ""News"" section",2019,10,1,6,2,"UI tests of the ""News"" section",871,0.000000,0.076923
3,4,0.013658,900,5,0,1,819941,93,93,"UI tests of the ""Zones of discounts and surcha...",2019,10,1,6,2,"UI tests of the ""Zones of discounts and surcha...",871,2.000000,0.096154
4,4,0.013658,900,5,0,1,819938,93,93,"UI tests of the ""Chat with support"" section",2019,10,1,6,2,"UI tests of the ""Chat with support"" section Ac...",871,2.000000,0.115385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5311,2,0.002422,3600,48,0,0,688428,1,1,Delete old cms page,2020,6,12,19,5,Delete old cms page,3595,0.000000,5.180965
5312,3,0.209532,5700,48,0,0,688425,1,1,bugfix and testing,2020,6,12,19,5,bugfix and testing,5255,0.000000,5.187668
5313,2,0.002422,3600,56,0,0,670932,2,2,Set up a graph in neo4j by GitLab dataset,2020,6,5,9,5,Set up a graph in neo4j by GitLab dataset,3595,0.000000,4.000000
5314,0,0.343479,6300,56,0,0,670931,2,2,Parse a dataset from teacherly slack,2020,6,5,13,5,Parse a dataset from teacherly slack,7030,0.000000,6.000000


In [7]:
test

,id,Id_razn,mean_words_comment,count_comments,project_id,Mesto_in_project,Year,Month,Day,Hour,DayOfWeek,assignee_id,creator_id,summary_to_eng,Text_Summa
0,721915,0,0.0,0,13,8.612540,2020,11,13,13,5,365,208,UI components: Published date doesn't display ...,UI components: Published date doesn't display ...
1,721911,0,0.0,0,13,8.616352,2020,11,13,15,5,82,82,Move elasticsearch from spot instances to onde...,Move elasticsearch from spot instances to onde...
2,721907,0,12.0,1,13,8.620164,2020,11,15,12,7,82,193,Website: Cast teacherly - close site and put o...,Website: Cast teacherly - close site and put o...
3,721902,0,0.0,0,13,8.624929,2020,11,16,4,1,365,193,Need to fix mark up on mobile resolution,Need to fix mark up on mobile resolution
4,721885,0,12.0,1,13,8.641128,2020,11,17,8,2,365,365,[Markup] Select background modal,[Markup] Select background modal [https://www....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,679006,0,0.0,0,48,1.239946,2020,6,24,19,3,1,1,Add open graphs,Add open graphs
247,679000,0,0.0,0,48,1.260054,2020,6,24,20,3,1,1,Change url structure for blog posts,Change url structure for blog posts
248,678985,0,0.0,0,48,1.320375,2020,6,26,16,5,82,82,Check for errors when the packer builds at the...,Check for errors when the packer builds at the...
249,702376,0,0.0,0,48,10.000000,2020,11,5,15,4,1,1,ipphone problem: push & audit result,ipphone problem: push & audit result


In [8]:
tabliza

,claster_centr,Номер кластера
0,7031,0
1,2044,1
2,3681,2
3,5376,3
4,871,4


In [9]:
train.columns

Index(['Номер кластера исходный', 'Расстояние до центра кластера',
       'overall_worklogs', 'project_id', 'Id_razn', 'count_comments', 'id',
       'assignee_id', 'creator_id', 'summary_to_eng', 'Year', 'Month', 'Day',
       'Hour', 'DayOfWeek', 'Text_Summa', 'claster_centr_ishodn',
       'mean_words_comment', 'Mesto_in_project'],
      dtype='object')

In [10]:
# выбираем поля для нейросети СatBoost по бинарной классификации

X_train=train[['project_id', 'assignee_id', 'creator_id', 'summary_to_eng',
        'Year', 'Month', 'Day', 'Hour', 'DayOfWeek', 'Mesto_in_project',
       'Id_razn', 'mean_words_comment', 'count_comments', 'Text_Summa' ]]
y_train=train[['Номер кластера исходный']]


testnet=test[['project_id', 'assignee_id', 'creator_id', 'summary_to_eng',
        'Year', 'Month', 'Day', 'Hour', 'DayOfWeek','Mesto_in_project',
       'Id_razn', 'mean_words_comment', 'count_comments','Text_Summa' ]]

In [11]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5316 entries, 0 to 5315
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   project_id          5316 non-null   int64  
 1   assignee_id         5316 non-null   int64  
 2   creator_id          5316 non-null   int64  
 3   summary_to_eng      5316 non-null   object 
 4   Year                5316 non-null   int64  
 5   Month               5316 non-null   int64  
 6   Day                 5316 non-null   int64  
 7   Hour                5316 non-null   int64  
 8   DayOfWeek           5316 non-null   int64  
 9   Mesto_in_project    5316 non-null   float64
 10  Id_razn             5316 non-null   int64  
 11  mean_words_comment  5316 non-null   float64
 12  count_comments      5316 non-null   int64  
 13  Text_Summa          5316 non-null   object 
dtypes: float64(2), int64(10), object(2)
memory usage: 581.6+ KB


In [12]:
# Определяем веса для классов кластеров
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes = np.unique(y_train), y = np.ravel(y_train))
class_weights = dict(zip(classes, weights))
class_weights

{0: 1.2248847926267281,
 1: 0.8286827747466875,
 2: 0.8601941747572815,
 3: 1.1444564047362755,
 4: 1.0632}

In [ ]:
#Обучаем 

pool_train_solution = Pool(X_train, y_train, cat_features = ['project_id', 'assignee_id', 'creator_id','Year', 'Month', 'Day', 'Hour', 'DayOfWeek'], text_features = ['summary_to_eng', 'Text_Summa'])
pool_test_solution = Pool(testnet,cat_features = ['project_id', 'assignee_id', 'creator_id','Year', 'Month', 'Day', 'Hour', 'DayOfWeek'], text_features = ['summary_to_eng', 'Text_Summa'])

model_test = CatBoostClassifier(iterations=1000,  task_type='GPU',depth=12, verbose =True, random_seed=33,# learning_rate=0.015,
                               leaf_estimation_method= 'Gradient',bootstrap_type= 'Poisson',objective = 'MultiClass',eval_metric = 'Accuracy',loss_function='MultiClassOneVsAll', class_weights=class_weights)
model_test.fit(pool_train_solution)

0:	learn: 0.4069715	total: 480ms	remaining: 7m 59s
1:	learn: 0.5161227	total: 777ms	remaining: 6m 27s
2:	learn: 0.6036575	total: 1.08s	remaining: 5m 58s
3:	learn: 0.6397808	total: 1.36s	remaining: 5m 39s
4:	learn: 0.6709460	total: 1.76s	remaining: 5m 49s
5:	learn: 0.6821325	total: 2.08s	remaining: 5m 44s
6:	learn: 0.6947997	total: 2.36s	remaining: 5m 34s
7:	learn: 0.7013456	total: 2.67s	remaining: 5m 30s
8:	learn: 0.7090715	total: 2.96s	remaining: 5m 26s
9:	learn: 0.7102574	total: 3.24s	remaining: 5m 21s
10:	learn: 0.7285195	total: 3.54s	remaining: 5m 18s
11:	learn: 0.7393253	total: 3.86s	remaining: 5m 17s
12:	learn: 0.7493953	total: 4.42s	remaining: 5m 35s
13:	learn: 0.7626040	total: 4.95s	remaining: 5m 48s
14:	learn: 0.7581545	total: 5.41s	remaining: 5m 55s
15:	learn: 0.7809998	total: 5.97s	remaining: 6m 7s
16:	learn: 0.7940019	total: 6.48s	remaining: 6m 14s
17:	learn: 0.8010949	total: 7.07s	remaining: 6m 25s
18:	learn: 0.8098946	total: 7.61s	remaining: 6m 32s
19:	learn: 0.8164991	to

In [ ]:
# предварительный тест схождения на всем обучающем множестве
y_pred = model_test.predict(X_train)
y_pred

In [ ]:
print("Значение метрики f1 на трейне по кластерам: ", f1_score(train['Номер кластера исходный'], y_pred, average='macro', zero_division = 0))

In [ ]:
train

In [ ]:
# Проверим уровень схождения по r2 на трейне
train['Номер кластера']=y_pred
train=train.merge(tabliza, how="left", on="Номер кластера")
train['Номер кластера'].value_counts()

In [ ]:
train['Номер кластера исходный'].value_counts()

In [ ]:
train

In [ ]:
print("Значение метрики r2 на трейне по секундам: ", r2_score(train['claster_centr'], train['overall_worklogs']))

In [ ]:
#Получение ответов
y_pred_solution = model_test.predict(pool_test_solution)

In [ ]:
# Заводим результаты в тест
test['Номер кластера']=y_pred_solution
test['Номер кластера'].value_counts()

In [ ]:
# заводим центры кластеров в качестве ответа
test=test.merge(tabliza, how="left", on="Номер кластера")
test

In [ ]:
test_itog=test[['id','claster_centr']]
test_itog.columns=[['id','overall_worklogs']]
test_itog

In [ ]:
test_itog.to_csv('прогноз Пуассон аккураси сид 33 кластер 5 1000 вниз.csv', sep=',', index=None)